# Embedding a whole set of pdfs in an automatic way

In [1]:
#import necessary packages
import os
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import subprocess
import openai
openai.api_key = "XXXXX"
os.environ["OPENAI_API_KEY"] = openai.api_key

## Retrieving the original list of articles

In [50]:
# Read the XLSX file
dataframes = pd.read_excel('WhitepaperD21.xlsx', sheet_name=['assignments', 'reviews', 'dbs', 'dbs_urls', 'tools', 'NeDRexDB-sources'])

# Do further processing with the DataFrame
# For example, print the first few rows

dataframes['assignments']['open acces?'] = dataframes['assignments']['open acces?'].str.replace('WAHR', 'Yes')
dataframes['assignments']['open acces?'] = dataframes['assignments']['open acces?'].str.replace('FALSCH', 'No')
dataframes['assignments'] = dataframes['assignments'][dataframes['assignments']['open acces?'] == 'Yes']

collections = {
    'dbs' : set(dataframes['dbs']['Title'].to_list()),
'reviews' : set(dataframes['reviews']['Title'].to_list()),
'tools' : set(dataframes['tools']['Title'].to_list()),
    
}
# Initialize the 'collection' column to None
dataframes['assignments']['collection'] = None

# Check each collection and update 'collection' accordingly
for collection_name, titles in collections.items():
    # Update the 'collection' column based on whether the title is in the current collection's set of titles
    dataframes['assignments'].loc[dataframes['assignments']['Title'].isin(titles), 'collection'] = collection_name

# Resulting DataFrame
df = dataframes['assignments']
df

,Name,Title,Authors,Year,PMID,DOI,Reviewed,Was there a problem?,open acces?,Unnamed: 9,collection
0,Andreas Maier,HIPPIE v2.0: enhancing meaningfulness and reli...,"Alanis-Lobato, Gregorio and Andrade-Navarro, M...",2017.0,27794551.0,10.1093/nar/gkw985,1.0,NaN,Yes,NaN,dbs
1,Andreas Maier,QUADrATiC: scalable gene expression connectivi...,"O'Reilly, Paul G and Wen, Qing and Bankhead, P...",2016.0,27143038.0,10.1186/s12859-016-1062-1,1.0,NaN,Yes,NaN,tools
2,Andreas Maier,Drug-set enrichment analysis: a novel tool to ...,"Napolitano, Francesco and Sirci, Francesco and...",2016.0,26415724.0,10.1093/bioinformatics/btv536,1.0,NaN,Yes,NaN,tools
5,Andreas Maier,RegNetwork: an integrated database of transcri...,"Liu, Zhi-Ping and Wu, Canglin and Miao, Hongyu...",2015.0,26424082.0,10.1093/database/bav095,1.0,NaN,Yes,NaN,dbs
6,Andreas Maier,The Cancer Cell Line Encyclopedia enables pred...,"Barretina, Jordi and Caponigro, Giordano and S...",2012.0,22460905.0,10.1038/nature11003,1.0,NaN,Yes,NaN,dbs
...,...,...,...,...,...,...,...,...,...,...,...
375,Quirin Manz,Drug-repositioning approaches based on medical...,"Zamami, Yoshito and Hamano, Hirofumi and Niimu...",2021.0,34790124.0,10.3389/fphar.2021.752174,1.0,NaN,Yes,NaN,None
376,Quirin Manz,KG-Predict: A knowledge graph computational fr...,"Gao, Zhenxiang and Ding, Pingjian and Xu, Rong",2022.0,35840060.0,10.1016/j.jbi.2022.104133,1.0,NaN,Yes,NaN,tools
377,Quirin Manz,The Human Phenotype Ontology in 2021,"Köhler, Sebastian and Gargano, Michael and Mat...",2021.0,33264411.0,10.1093/nar/gkaa1043,1.0,NaN,Yes,NaN,dbs
378,Quirin Manz,Evaluation of connectivity map shows limited r...,"Lim, Nathaniel and Pavlidis, Paul",2021.0,34475469.0,10.1038/s41598-021-97005-z,1.0,NaN,Yes,NaN,tools


In [51]:
file_list = os.listdir('all_papers/')
# Create a dictionary to map file names to paths
file_path_dict = {}
for file_path in file_list:
    file_name = file_path.split('/')[-1]
    file_path_dict[file_name] = f"all_papers/{file_path}"
    

# Function to find the closest matching file name
def get_closest_match(title):
    closest_match = process.extractOne(title, file_path_dict.keys(), scorer=fuzz.ratio)
    if closest_match[1] >= 50:  # Adjust the threshold as per your requirements
        return file_path_dict[closest_match[0]]
    else:
        return None
df['pdf_path'] = df['Title'].apply(get_closest_match)
df

,Name,Title,Authors,Year,PMID,DOI,Reviewed,Was there a problem?,open acces?,Unnamed: 9,collection,pdf_path
0,Andreas Maier,HIPPIE v2.0: enhancing meaningfulness and reli...,"Alanis-Lobato, Gregorio and Andrade-Navarro, M...",2017.0,27794551.0,10.1093/nar/gkw985,1.0,NaN,Yes,NaN,dbs,all_papers/Alanis-Lobato et al. 2017 - HIPPIE ...
1,Andreas Maier,QUADrATiC: scalable gene expression connectivi...,"O'Reilly, Paul G and Wen, Qing and Bankhead, P...",2016.0,27143038.0,10.1186/s12859-016-1062-1,1.0,NaN,Yes,NaN,tools,all_papers/O_Reilly et al. 2016 - QUADrATiC - ...
2,Andreas Maier,Drug-set enrichment analysis: a novel tool to ...,"Napolitano, Francesco and Sirci, Francesco and...",2016.0,26415724.0,10.1093/bioinformatics/btv536,1.0,NaN,Yes,NaN,tools,all_papers/Napolitano et al. 2016 - Drug-set e...
5,Andreas Maier,RegNetwork: an integrated database of transcri...,"Liu, Zhi-Ping and Wu, Canglin and Miao, Hongyu...",2015.0,26424082.0,10.1093/database/bav095,1.0,NaN,Yes,NaN,dbs,all_papers/Liu et al. 2015 - RegNetwork - an i...
6,Andreas Maier,The Cancer Cell Line Encyclopedia enables pred...,"Barretina, Jordi and Caponigro, Giordano and S...",2012.0,22460905.0,10.1038/nature11003,1.0,NaN,Yes,NaN,dbs,all_papers/Barretina et al. 2012 - The Cancer ...
...,...,...,...,...,...,...,...,...,...,...,...,...
375,Quirin Manz,Drug-repositioning approaches based on medical...,"Zamami, Yoshito and Hamano, Hirofumi and Niimu...",2021.0,34790124.0,10.3389/fphar.2021.752174,1.0,NaN,Yes,NaN,None,all_papers/Zamami et al. 2021 - Drug-Repositio...
376,Quirin Manz,KG-Predict: A knowledge graph computational fr...,"Gao, Zhenxiang and Ding, Pingjian and Xu, Rong",2022.0,35840060.0,10.1016/j.jbi.2022.104133,1.0,NaN,Yes,NaN,tools,all_papers/Gao et al. 2022 - KG-Predict - A kn...
377,Quirin Manz,The Human Phenotype Ontology in 2021,"Köhler, Sebastian and Gargano, Michael and Mat...",2021.0,33264411.0,10.1093/nar/gkaa1043,1.0,NaN,Yes,NaN,dbs,all_papers/Köhler et al. 2021 - The Human Phe...
378,Quirin Manz,Evaluation of connectivity map shows limited r...,"Lim, Nathaniel and Pavlidis, Paul",2021.0,34475469.0,10.1038/s41598-021-97005-z,1.0,NaN,Yes,NaN,tools,all_papers/Lim and Pavlidis 2021 - Evaluation ...


In [52]:
df['pdf_path'].isna().sum()
    

0

## Loading pdfs and adding metadata

In [53]:
df.loc[df.Title == 'Cancer driver drug interaction explorer', 'pdf_path'] = "/Users/fernando/Documents/Research/ChatGPT_REPO4EU/data/d23_repo4eu/data/all_papers/Hartung et al. 2022 - Cancer driver drug interaction explorer.pdf"
df.loc[df.Title == 'Drug Repositioning and Target Finding Based on Clinical Evidence', 'pdf_path']  = "/Users/fernando/Documents/Research/ChatGPT_REPO4EU/data/d23_repo4eu/data/all_papers/Kaneko and Nagashima 2020 - Drug Repositioning and Target Finding Based on Clinical Evidence.pdf"

In [54]:
df.head()

,Name,Title,Authors,Year,PMID,DOI,Reviewed,Was there a problem?,open acces?,Unnamed: 9,collection,pdf_path
0,Andreas Maier,HIPPIE v2.0: enhancing meaningfulness and reli...,"Alanis-Lobato, Gregorio and Andrade-Navarro, M...",2017.0,27794551.0,10.1093/nar/gkw985,1.0,NaN,Yes,NaN,dbs,all_papers/Alanis-Lobato et al. 2017 - HIPPIE ...
1,Andreas Maier,QUADrATiC: scalable gene expression connectivi...,"O'Reilly, Paul G and Wen, Qing and Bankhead, P...",2016.0,27143038.0,10.1186/s12859-016-1062-1,1.0,NaN,Yes,NaN,tools,all_papers/O_Reilly et al. 2016 - QUADrATiC - ...
2,Andreas Maier,Drug-set enrichment analysis: a novel tool to ...,"Napolitano, Francesco and Sirci, Francesco and...",2016.0,26415724.0,10.1093/bioinformatics/btv536,1.0,NaN,Yes,NaN,tools,all_papers/Napolitano et al. 2016 - Drug-set e...
5,Andreas Maier,RegNetwork: an integrated database of transcri...,"Liu, Zhi-Ping and Wu, Canglin and Miao, Hongyu...",2015.0,26424082.0,10.1093/database/bav095,1.0,NaN,Yes,NaN,dbs,all_papers/Liu et al. 2015 - RegNetwork - an i...
6,Andreas Maier,The Cancer Cell Line Encyclopedia enables pred...,"Barretina, Jordi and Caponigro, Giordano and S...",2012.0,22460905.0,10.1038/nature11003,1.0,NaN,Yes,NaN,dbs,all_papers/Barretina et al. 2012 - The Cancer ...


In [55]:
from langchain.document_loaders import PyPDFLoader
from tqdm import tqdm

In [56]:
list_of_documents = []

for index, row in tqdm(df.iterrows()):
    if "data" not in row['pdf_path']:
        filepath = "/Users/fernando/Documents/Research/ChatGPT_REPO4EU/data/d23_repo4eu/data/" + row['pdf_path']
    else:
        filepath = row['pdf_path']
    
    try:
        loader = PyPDFLoader(filepath)
        document = loader.load_and_split()
        
        for i in range(len(document)):
            document[i].page_content = document[i].page_content.split("\nReferences\n")[0]
            document[i].metadata.update(row.to_dict())
        list_of_documents.extend(document)
    except:
        print(f"article not found!\n{row['Title']}")

19it [00:12,  2.94it/s]Multiple definitions in dictionary at byte 0x26cf2 for key /MediaBox
Multiple definitions in dictionary at byte 0x2713f for key /MediaBox
Multiple definitions in dictionary at byte 0x272fd for key /MediaBox
Multiple definitions in dictionary at byte 0x2753e for key /MediaBox
Multiple definitions in dictionary at byte 0x276b7 for key /MediaBox
Multiple definitions in dictionary at byte 0x278e8 for key /MediaBox
Multiple definitions in dictionary at byte 0x27a6e for key /MediaBox
Multiple definitions in dictionary at byte 0x27c84 for key /MediaBox
Multiple definitions in dictionary at byte 0x27e6d for key /MediaBox
Multiple definitions in dictionary at byte 0x280a6 for key /MediaBox
Multiple definitions in dictionary at byte 0x2832f for key /MediaBox
Multiple definitions in dictionary at byte 0x285c0 for key /MediaBox
262it [04:58,  2.40it/s]Multiple definitions in dictionary at byte 0xf25e for key /MediaBox
Multiple definitions in dictionary at byte 0xf51d for key

article not found!
Advancing drug repurposing research: Trends, collaborative networks, innovation and knowledge leaders


285it [05:07,  1.08s/it]


### Embedding and vector-storing

In [9]:
sublist_docs = list_of_documents[:5]

#### FAISS implementation

In [57]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

my_embedding_model = OpenAIEmbeddings()

vectordb = FAISS.from_documents(
    documents=list_of_documents,
    embedding=my_embedding_model)

In [64]:
import faiss
import pickle

def write_file(filename, content):
    with open(filename, 'wb') as file:
        file.write(content)


def read_file(filename):
    with open(filename, 'rb') as file:
        return file.read()

def store_index_in_db(index, name):
    faiss.write_index(index.index, "docs.index")
    # Open the file and dump to local storage
    write_file(f"{name}.index", read_file("docs.index"))
    index.index = None
    write_file(f"{name}.pkl", pickle.dumps(index))


def load_index_from_db(index_name):
    findex = read_file(f"{index_name}.index")

    write_file("docs.index", findex)
    index = faiss.read_index("docs.index")
    VectorDB = pickle.loads(read_file(f"{index_name}.pkl"))
    VectorDB.index = index

    return VectorDB


store_index_in_db(vectordb, "repo4euD21openaccess")

RuntimeError: Error in void faiss::write_index(const faiss::Index *, faiss::IOWriter *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/impl/index_write.cpp:821: don't know how to serialize this type of index

In [59]:
index = load_index_from_db("repo4euD21openaccess")


In [60]:
print(len(list_of_documents))
print(len(index.docstore._dict))

5761
5761


## Similarity searches

In [41]:
question = "what is drug repurposing?"
docs = index.similarity_search(question, k=3)
docs

[Document(page_content='3 \n 1. Introduction  1 \nDrug repurposing (DR) is the process of identifying new therapeutic applications for existing drugs 2 \n[1]. Over the past few years, pharmaceutical industries have hugely invested in the repositioning  of 3 \napproved  and withdrawn  drugs  as traditional  drug development  is an extremely  expensive, 4 \nlaborious, time -consuming, and highly failure -prone avenue  [2-6]. DR especially finds its 5 \napplication in rare and  neglected diseases where there are very few or no drugs available for 6 \ntreatment  [7]. The US -FDA has provided a list of approved pharmaceuticals that can be promising 7 \ndrug candidates for  repurposing in rare disea ses [8]. Nonetheless, it has always been an endeavour  to 8 \nidentify drugs  that are equipotent  to such orphan  drugs.  DR also finds  its application  in 9 \ninfectious  diseases such  as tuberculosis [ 9-15], HIV and other communicable diseases where multi - 10 \ndrug resistance is a  major 

## Question answering

In [63]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
import os 
import openai


os.environ["OPENAI_API_KEY"] = openai.api_key


my_embedding_model = OpenAIEmbeddings()
vectordb2 = FAISS.load_local("repo4euD21openaccess", my_embedding_model)



RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char *) at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/impl/io.cpp:68: Error: 'f' failed: could not open repo4euD21openaccess/index.faiss for reading: No such file or directory

In [66]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# Build prompt

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template("""You are great at answering questions about drug repurposing in a concise\
                                                    and easy to understand manner. \
                                                    When you don't know the answer to a question you admit that you don't know\
                                                    Here is a question:\
                                                    {user_prompt}""")  
    ],
    input_variables=["user_prompt"])


label_query = prompt.format_prompt(user_prompt="Who is Yajie Meng?")



In [67]:

# Create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature = 0.0, model='gpt-3.5-turbo'),
                                  chain_type="stuff",
                                  retriever=index.as_retriever(),
                                  return_source_documents=True,
                                  verbose=True)

# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['DOI'])
        
# Question
query = "Who is Yajie Meng?"
llm_response = qa_chain(prompt.format_prompt(user_prompt=query).to_string())
process_llm_response(llm_response)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new RetrievalQA chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error

KeyboardInterrupt: 